Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [3]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
print(train.shape, test.shape)
train.head()

(4087, 3) (1022, 2)


,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [4]:
test.head()

,id,description
0,3461,\nStyle: Speyside single malt scotch Color: Wa...
1,2604,"\nVery bright and lively, with a nice balance ..."
2,3341,\nA new oloroso-forward Chivas positioned to s...
3,3764,\nAged in bourbon casks and then enhanced in R...
4,2306,\nThere is a freshness to the wood on the nose...


In [5]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(train, 
                              test_size=0.2,
                              stratify=train['ratingCategory'])
print(train.shape, val.shape)

(3269, 3) (818, 3)


In [6]:
train.ratingCategory.value_counts(normalize=True)

1    0.704803
0    0.279290
2    0.015907
Name: ratingCategory, dtype: float64

In [7]:
val.ratingCategory.value_counts(normalize=True)

1    0.705379
0    0.278729
2    0.015892
Name: ratingCategory, dtype: float64

### Define Pipeline Components

In [8]:
import re

# Optional: Use a custom tokenizer
def my_tokenizer(text):
    clean_text = re.sub('[^a-zA-Z ]', '', text)
    tokens = clean_text.lower().split()
    return tokens

In [9]:
X = train['description']
Y = train['ratingCategory']
x_test = test['description']

In [10]:
# vect = TfidfVectorizer()
# clf = RandomForestClassifier()
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

vect = TfidfVectorizer(tokenizer=my_tokenizer)
clf = RandomForestClassifier()

from sklearn.pipeline import FeatureUnion, Pipeline

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 
## PARAMETERS

In [13]:
# Small grid search to reduce runtime
parameters = {
    # 'vect__min_df': (1, 2),
    # 'clf__C': (0.1, 0.2),
    'clf__n_estimators': [10],
    'vect__max_df': [.95],
    'vect__max_features': [50],
    'vect__min_df': [.01],
    'clf__max_depth':(20, 10)
}
# parameters = {
#     'vect__max_df': (0.75, 1.0),
#     'clf__max_depth':(20, 30)
# }
# # {'clf__max_depth': 20, 'vect__max_df': 0.75} 0.7078619848105435

In [14]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(pipe, parameters, cv=2, n_jobs=-1, verbose=1)
grid_search.fit(X, Y)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(tokenizer=<function my_tokenizer at 0x000002184881DF30>)),
                                       ('clf', RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'clf__max_depth': (20, 10), 'clf__n_estimators': [10],
                         'vect__max_df': [0.95], 'vect__max_features': [50],
                         'vect__min_df': [0.01]},
             verbose=1)

In [15]:
grid_search.best_params_

{'clf__max_depth': 10,
 'clf__n_estimators': 10,
 'vect__max_df': 0.95,
 'vect__max_features': 50,
 'vect__min_df': 0.01}

In [16]:
grid_search.best_score_

0.7044971346651245

### Evaluate on Validation Data

In [17]:
from sklearn.metrics import accuracy_score

# Evaluate on test data
y_val = grid_search.predict(val.description)
accuracy_score(val.ratingCategory, y_val)

0.7004889975550123

In [18]:
from sklearn.metrics import classification_report

print(classification_report(val.ratingCategory, y_val))

              precision    recall  f1-score   support

           0       0.43      0.07      0.12       228
           1       0.71      0.97      0.82       577
           2       0.00      0.00      0.00        13

    accuracy                           0.70       818
   macro avg       0.38      0.35      0.31       818
weighted avg       0.62      0.70      0.61       818



C:\Users\somer\miniconda3\envs\DeepLearning\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\somer\miniconda3\envs\DeepLearning\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\somer\miniconda3\envs\DeepLearning\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [19]:
# Predictions on test sample
pred = grid_search.predict(x_test)

In [20]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [21]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [ ]:
subNumber = 2

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 80% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [ ]:
vect = TfidfVectorizer(stop_words='english', 
                       ngram_range=(1,2),
                       min_df=2,
                       max_df=0.3)
svd = TruncatedSVD(n_components=..., 
                   algorithm='randomized', 
                   n_iter=10)
svm = LinearSVC(C=0.5, 
                penalty='l2')

pipe = Pipeline([
    ('vect', vect),      # TF-IDF Vectorizer
    ('svd', svd),        # LSI (SVD Dim Reduction)
    ('clf', svm)         # LinearSVC Classifier
])
pipe.fit(...)

### (Skip this Section for now) Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

### Evaluate on Validation Data

In [ ]:
from sklearn.metrics import accuracy_score

# Evaluate on test data
y_val = pipe.predict(val.description)
accuracy_score(val.ratingCategory, y_val)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(val.ratingCategory, y_val))

### Make a Submission File

In [ ]:
# Predictions on test sample
pred = pipe.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [ ]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [ ]:
# Continue Word Embedding Work Here


### Make a Submission File

In [ ]:
# Predictions on test sample
pred = ...predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 80% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?